# Tracking electoral con ponderación censal (Argentina)Notebook de seguimiento para simular un tracking electoral con datos ficticios de tres encuestas mensuales. El flujo carga múltiples CSV, limpia los datos, pondera la muestra con referencias del Censo 2022 de Argentina (sexos, grupos de edad y estrato socioeconómico) y grafica la evolución de la imagen de *Candidato A* y de la intención de voto para tres candidatos.

## Librerías y configuración de gráficos

In [ ]:
import mathfrom pathlib import Pathfrom typing import Dict, Iterableimport matplotlib.pyplot as pltimport pandas as pdimport seaborn as snssns.set_theme(style="whitegrid")plt.rcParams["figure.figsize"] = (10, 6)

## Distribución objetivo (Censo 2022)A modo de ejemplo, usamos una distribución simplificada inspirada en el Censo 2022 (valores porcentuales aproximados). Se pueden reemplazar por tablas oficiales de INDEC si se cuenta con el detalle por sexo, grupo etario y estrato socioeconómico derivado de la canasta básica.

In [ ]:
# Proporciones aproximadas (suman 1) para cruces sexo x grupo de edad x estrato.census_distribution = pd.DataFrame(    [        {"sexo": "Femenino", "grupo_edad": "18-29", "estrato": "Bajo", "prop_objetivo": 0.055},        {"sexo": "Femenino", "grupo_edad": "18-29", "estrato": "Medio", "prop_objetivo": 0.060},        {"sexo": "Femenino", "grupo_edad": "18-29", "estrato": "Alto", "prop_objetivo": 0.015},        {"sexo": "Femenino", "grupo_edad": "30-44", "estrato": "Bajo", "prop_objetivo": 0.065},        {"sexo": "Femenino", "grupo_edad": "30-44", "estrato": "Medio", "prop_objetivo": 0.070},        {"sexo": "Femenino", "grupo_edad": "30-44", "estrato": "Alto", "prop_objetivo": 0.020},        {"sexo": "Femenino", "grupo_edad": "45-59", "estrato": "Bajo", "prop_objetivo": 0.050},        {"sexo": "Femenino", "grupo_edad": "45-59", "estrato": "Medio", "prop_objetivo": 0.055},        {"sexo": "Femenino", "grupo_edad": "45-59", "estrato": "Alto", "prop_objetivo": 0.020},        {"sexo": "Femenino", "grupo_edad": "60+", "estrato": "Bajo", "prop_objetivo": 0.040},        {"sexo": "Femenino", "grupo_edad": "60+", "estrato": "Medio", "prop_objetivo": 0.045},        {"sexo": "Femenino", "grupo_edad": "60+", "estrato": "Alto", "prop_objetivo": 0.015},        {"sexo": "Masculino", "grupo_edad": "18-29", "estrato": "Bajo", "prop_objetivo": 0.055},        {"sexo": "Masculino", "grupo_edad": "18-29", "estrato": "Medio", "prop_objetivo": 0.060},        {"sexo": "Masculino", "grupo_edad": "18-29", "estrato": "Alto", "prop_objetivo": 0.015},        {"sexo": "Masculino", "grupo_edad": "30-44", "estrato": "Bajo", "prop_objetivo": 0.065},        {"sexo": "Masculino", "grupo_edad": "30-44", "estrato": "Medio", "prop_objetivo": 0.070},        {"sexo": "Masculino", "grupo_edad": "30-44", "estrato": "Alto", "prop_objetivo": 0.020},        {"sexo": "Masculino", "grupo_edad": "45-59", "estrato": "Bajo", "prop_objetivo": 0.050},        {"sexo": "Masculino", "grupo_edad": "45-59", "estrato": "Medio", "prop_objetivo": 0.055},        {"sexo": "Masculino", "grupo_edad": "45-59", "estrato": "Alto", "prop_objetivo": 0.020},        {"sexo": "Masculino", "grupo_edad": "60+", "estrato": "Bajo", "prop_objetivo": 0.040},        {"sexo": "Masculino", "grupo_edad": "60+", "estrato": "Medio", "prop_objetivo": 0.045},        {"sexo": "Masculino", "grupo_edad": "60+", "estrato": "Alto", "prop_objetivo": 0.015},    ])assert math.isclose(census_distribution["prop_objetivo"].sum(), 1.0, rel_tol=1e-6), "La distribución objetivo debe sumar 1"

## Utilidades de limpieza y ponderación

In [ ]:
DATA_DIR = Path("data")OUTPUT_DIR = Path("outputs")OUTPUT_DIR.mkdir(exist_ok=True)def normalizar_estrato(valor: str) -> str:    if pd.isna(valor):        return None    valor = valor.strip().lower()    if valor.startswith("b"):        return "Bajo"    if valor.startswith("m"):        return "Medio"    if valor.startswith("a"):        return "Alto"    return Nonedef normalizar_sexo(valor: str) -> str:    if pd.isna(valor):        return None    valor = valor.strip().lower()    if valor.startswith("f"):        return "Femenino"    if valor.startswith("m"):        return "Masculino"    return Nonedef agrupar_edad(edad: float) -> str:    if pd.isna(edad):        return None    edad = float(edad)    if edad < 18:        return None    if edad <= 29:        return "18-29"    if edad <= 44:        return "30-44"    if edad <= 59:        return "45-59"    return "60+"def cargar_encuestas(ruta: Path) -> pd.DataFrame:    archivos = sorted(ruta.glob("encuestas_*.csv"))    if not archivos:        raise FileNotFoundError("No se encontraron archivos encuestas_*.csv en el directorio de datos")    frames = []    for archivo in archivos:        df = pd.read_csv(archivo, parse_dates=["Fecha"], dayfirst=False)        df["archivo"] = archivo.name        frames.append(df)    return pd.concat(frames, ignore_index=True)def limpiar_encuestas(df: pd.DataFrame) -> pd.DataFrame:    base = df.copy()    base["Estrato"] = base["Estrato"].apply(normalizar_estrato)    base["Sexo"] = base["Sexo"].apply(normalizar_sexo)    base["grupo_edad"] = base["Edad"].apply(agrupar_edad)    base["Voto"] = base["Voto"].str.strip().str.title()    base["Voto Anterior"] = base["Voto Anterior"].str.strip().str.title()    base["Imagen del Candidato"] = pd.to_numeric(base["Imagen del Candidato"], errors="coerce")    # Descarta registros incompletos para la ponderación    base = base.dropna(subset=["Estrato", "Sexo", "grupo_edad", "Imagen del Candidato", "Voto"])    return basedef calcular_pesos_muestrales(df: pd.DataFrame, distribucion_objetivo: pd.DataFrame) -> pd.DataFrame:    claves = ["Sexo", "grupo_edad", "Estrato"]    df_claves = df.rename(columns={"Sexo": "sexo", "Estrato": "estrato"})    muestra = df_claves.groupby(["sexo", "grupo_edad", "estrato"]).size().reset_index(name="n_muestra")    muestra["prop_muestra"] = muestra["n_muestra"] / muestra["n_muestra"].sum()    mergeado = distribucion_objetivo.merge(muestra, how="left", on=["sexo", "grupo_edad", "estrato"])    mergeado["prop_muestra"].fillna(0, inplace=True)    mergeado["peso"] = mergeado.apply(        lambda fila: fila["prop_objetivo"] / fila["prop_muestra"] if fila["prop_muestra"] > 0 else 0,        axis=1,    )    pesos = mergeado[["sexo", "grupo_edad", "estrato", "peso"]]    df_pesado = df_claves.merge(pesos, how="left", on=["sexo", "grupo_edad", "estrato"])    df_pesado["peso"].fillna(0, inplace=True)    return df_pesado.rename(columns={"sexo": "Sexo", "estrato": "Estrato"})

## Cargar y preparar la base unificada

In [ ]:
encuestas_raw = cargar_encuestas(DATA_DIR)encuestas_limpias = limpiar_encuestas(encuestas_raw)encuestas_pesadas = calcular_pesos_muestrales(encuestas_limpias, census_distribution)encuestas_pesadas.head()

## Funciones de agregación y ventana móvil

In [ ]:
def media_ponderada(df: pd.DataFrame, valor_col: str, peso_col: str) -> float:    if df[peso_col].sum() == 0:        return float("nan")    return (df[valor_col] * df[peso_col]).sum() / df[peso_col].sum()def evolucion_imagen(df: pd.DataFrame, candidato_objetivo: str, ventana: int = 2) -> pd.DataFrame:    filtro = df[df["Voto"] == candidato_objetivo]    serie = (        filtro.groupby("Fecha")        .apply(lambda x: media_ponderada(x, "Imagen del Candidato", "peso"))        .reset_index(name="imagen_media")        .sort_values("Fecha")    )    serie["imagen_suavizada"] = serie["imagen_media"].rolling(window=ventana, min_periods=1).mean()    return seriedef evolucion_voto(df: pd.DataFrame, ventana: int = 2) -> pd.DataFrame:    totales = df.groupby(["Fecha", "Voto"]).agg({"peso": "sum"}).reset_index()    totales["peso_total"] = totales.groupby("Fecha")["peso"].transform("sum")    totales["voto_pct"] = (totales["peso"] / totales["peso_total"]) * 100    totales = totales.sort_values("Fecha")    totales["voto_suavizado"] = (        totales.groupby("Voto")["voto_pct"].transform(lambda s: s.rolling(window=ventana, min_periods=1).mean())    )    return totales

## Calcular series de tiempo ponderadas

In [ ]:
CANDIDATO_OBJETIVO = "Candidato A"VENTANA = 2serie_imagen = evolucion_imagen(encuestas_pesadas, candidato_objetivo=CANDIDATO_OBJETIVO, ventana=VENTANA)serie_voto = evolucion_voto(encuestas_pesadas, ventana=VENTANA)serie_imagen

## Gráficos: imagen del candidato y voto por candidato

In [ ]:
fig1, ax1 = plt.subplots()sns.lineplot(data=serie_imagen, x="Fecha", y="imagen_media", marker="o", label="Media ponderada", ax=ax1)sns.lineplot(data=serie_imagen, x="Fecha", y="imagen_suavizada", marker="o", label=f"Suavizada ({VENTANA})", ax=ax1)ax1.set_title(f"Evolución de la imagen de {CANDIDATO_OBJETIVO}")ax1.set_ylabel("Imagen (0-100)")ax1.set_xlabel("Fecha de campo")fig1.autofmt_xdate()img_path = OUTPUT_DIR / "imagen_candidato.png"fig1.savefig(img_path, dpi=150, bbox_inches="tight")img_path

In [ ]:
fig2, ax2 = plt.subplots()sns.lineplot(data=serie_voto, x="Fecha", y="voto_pct", hue="Voto", style="Voto", marker="o", ax=ax2)sns.lineplot(data=serie_voto, x="Fecha", y="voto_suavizado", hue="Voto", style=True, dashes=True, legend=False, ax=ax2)ax2.set_title("Evolución de intención de voto (ponderado y suavizado)")ax2.set_ylabel("Porcentaje")ax2.set_xlabel("Fecha de campo")fig2.autofmt_xdate()voto_path = OUTPUT_DIR / "intencion_voto.png"fig2.savefig(voto_path, dpi=150, bbox_inches="tight")voto_path

## Uso en VSCode1. Abre este repositorio en VSCode y abre `tracking_electoral.ipynb`.2. Instala las dependencias en un entorno de Python 3.10+: `pip install pandas matplotlib seaborn`.3. Ejecuta las celdas en orden. Las gráficas se guardan en `outputs/imagen_candidato.png` e `outputs/intencion_voto.png` para referencia rápida.4. Para actualizar la simulación, agrega nuevos CSV en `data/` siguiendo el patrón `encuestas_YYYY-MM.csv` y vuelve a ejecutar las celdas.